In [1]:
%matplotlib inline

In [2]:
# %load_ext Cython

In [3]:
from __future__ import division
import networkx as nx
import random
import math
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pickle as pl
import sys

from pandas import Series, DataFrame
import pandas as pd
# cimport numpy as np

In [4]:
def figsize(scale):
    fig_width_pt = 395.15166                         # Get this from LaTeX using \the\textwidth
    inches_per_pt = 1.0/72.27                       # Convert pt to inch
    golden_mean = (np.sqrt(5.0)-1.0)/2.0            # Aesthetic ratio (you could change this)
    fig_width = fig_width_pt*inches_per_pt*scale    # width in inches
    fig_height = fig_width*golden_mean              # height in inches
    fig_size = [fig_width,fig_height]
    return fig_size
pgf_with_latex = {                      # setup matplotlib to use latex for output
    "pgf.texsystem": "pdflatex",        # change this if using xetex or lautex
    "text.usetex": True,                # use LaTeX to write all text
    "font.family": "serif",
    "font.serif": [],                   # blank entries should cause plots to inherit fonts from the document
    "font.sans-serif": [],
    "font.monospace": [],
    "axes.labelsize": 10,               # LaTeX default is 10pt font.
    "font.size": 8,
    "legend.fontsize": 8,               # Make the legend/label fonts a little smaller
    "xtick.labelsize": 8,
    "ytick.labelsize": 8,
    "figure.figsize": figsize(0.9),     # default fig size of 0.9 textwidth
    "pgf.preamble": [
        r"\usepackage[utf8x]{inputenc}",    # use utf8 fonts becasue your computer can handle it :)
        r"\usepackage[T1]{fontenc}",        # plots will be generated using this preamble
        ]
    }
matplotlib.rcParams.update(pgf_with_latex)

cust_color=["#e41a1c", "#377eb8", "#4daf4a", "#984ea3", "#ff7f00", "#ffff33", "#a65628", "#f781bf"]
#these colors come from colorbrewer2.org. Each is an RGB triplet
dark2_colors = [(0.10588235294117647, 0.6196078431372549, 0.4666666666666667),
                (0.8509803921568627, 0.37254901960784315, 0.00784313725490196),
                (0.4588235294117647, 0.4392156862745098, 0.7019607843137254),
                (0.9058823529411765, 0.1607843137254902, 0.5411764705882353),
                (0.4, 0.6509803921568628, 0.11764705882352941),
                (0.9019607843137255, 0.6705882352941176, 0.00784313725490196),
                (0.6509803921568628, 0.4627450980392157, 0.11372549019607843),
                (0.4, 0.4, 0.4)]

In [5]:
inline_rc = dict(matplotlib.rcParams) #Sepcific for Matploblib inline Ipython notebook
matplotlib.rcParams['savefig.dpi'] = 125
matplotlib.rcParams['text.latex.preamble']=[r"\usepackage{amsmath,amssymb,amsfonts}"]

# Parameters

In [35]:
##### Parameters
G_i = 1 # 1 for Friendster, 2 for Les miserables network
if G_i == 1:
    tour_N = 25 #5
    tour_c = 0
    deg_def = [50]
    flag_fn = 1 # 1 for g(v) = d(v) > 25, 4 for avg clustering coeff
if G_i == 2:
    tour_N =  10 #5
    tour_c = 0
    deg_def = [10,4]
    flag_fn = 1 # 1 for g(v) = d(v) > 10, 2 for g(v) = d(v) < 4, 3 for avg degree, 4 for avg clustering coeff
    asy_var_MH_lesmis = [4.4154,14.93508,1.204e+03]
    asy_var_RDS_lesmis = [1.1084,3.3240,272.7649]

max_B = 10000 # maximum budget
no_runs = 100 # no. of runs to average
MH_do = 0 # Flag for doing MH simulation
RDS_do = 1 # Flag for doing RDS simulation
RL_MH_do = 1 # Flag for doing RL simulation with MH sampling
RL_RW_do = 1 # Flag for doing RL simulation with RWRDS (actually unbiased) sampling

SHOW_VAR_CONVG = 0
ABS_ERROR = 1

# Functions

In [36]:
def node_fn(node):
    if flag_fn == 1:
        temp = int(G.degree(node)>deg_def[0])
    elif flag_fn == 2:
        temp = int(G.degree(node)<deg_def[1])
    elif flag_fn == 3:
        temp = G.degree(node)
    elif flag_fn == 4:
        deg = G.degree(node)
        if deg < 2:
            temp = 0
        else:
            temp = 2*nx.triangles(G,node)/(deg*(deg-1))
    else:
        print("Not a defined a function")
        sys.exit(0)
    return temp

In [37]:
def MH_sampling(G,B):
    est_MH= []
    est_MH_t = 0
    sample = np.random.choice(G.nodes())
    est_MH_t += node_fn(sample)
    est_MH.append(est_MH_t)
    for ii in range(2,B+1):
        # print "MH_sample: ",ii
        neighbors = nx.neighbors(G,sample)
        sample_t = np.random.choice(neighbors)
        if np.random.rand() <= (G.degree(sample)/G.degree(sample_t)):
            sample = sample_t

        est_MH_t += node_fn(sample)
        est_MH.append(est_MH_t/ii)
    return np.array(est_MH)

In [38]:
def RDS_sampling(G,B):
    est_RW = []
    est_RW_t1 = 0
    est_RW_t2 = 0
    sample = np.random.choice(G.nodes())
    deg_pr_sent = G.degree(sample)
    est_RW_t1 += node_fn(sample)/deg_pr_sent
    est_RW_t2 += 1/deg_pr_sent
    est_RW.append(est_RW_t1/est_RW_t2)
    for ii in range(2,B+1):
        # print "rds_sample: ",ii
        neighbors = nx.neighbors(G,sample)
        sample = random.choice(neighbors)

        deg_pr_sent = G.degree(sample)
        est_RW_t1 += node_fn(sample)/deg_pr_sent
        est_RW_t2 += 1/deg_pr_sent
        est_RW.append(est_RW_t1/est_RW_t2)
    return np.array(est_RW)

## RL-MH estimate

In [39]:
def RL_estimate_MH(G,super_node,max_B,B_arr,G_no_nodes,techq,tour_N,tour_c):
    """
    1. Returns estimate from tours
    2. function  fn: f(D_t) = 1 if D_t > deg_def

    Parameters
    ----------
    G:            - networkx graph
    super_node	  - node set  for "super node"
    no_tours	  - number of tours
    """
    size_super_node = len(super_node)
    V = np.zeros((size_super_node,max_B+1))
    V_def = np.zeros(max_B)
    Z_def = 10
    R=[]
    xi_arr = []
    xi_sum = 0
    tour_i = 1
    while xi_sum <= max_B:
        # print "RL tour:", tour_i
        R_k = 0
        Z_i = random.randint(0,size_super_node-1)
        # Z = super_node[Z_i]
        # neighbors = nx.neighbors(G,Z)
        # sample = random.choice(neighbors)
        sample = super_node[Z_i]
        neighbors = nx.neighbors(G,sample)
        sample_t = np.random.choice(neighbors)
        if np.random.rand() <= (G.degree(sample)/G.degree(sample_t)):
            sample = sample_t
        xi = 1
        while sample not in super_node:
            R_k += node_fn(sample)
            xi += 1
            neighbors = nx.neighbors(G,sample)
            sample_t = random.choice(neighbors)
            if np.random.rand() <= (G.degree(sample)/G.degree(sample_t)):
                sample = sample_t
        R_k += node_fn(sample)
        R.append(R_k)
        xi_sum += xi
        # xi_arr maintains the renewal epochs \xi(1), \xi(1)+\xi(2), ....
        xi_arr.append(xi_sum)

        V[:,tour_i] = V[:,(tour_i-1)]

        Z_f = super_node.index(sample)
        if techq == 1:
            V_def[tour_i-1] = V[Z_def,tour_i-1]
            V[Z_i][tour_i] += (1 / (np.ceil(tour_i/tour_N)+tour_c))* \
                            (R_k - V_def[tour_i-1]*xi + V[Z_f][tour_i-1] - V[Z_i][tour_i-1])
        elif techq == 2:
            V_def[tour_i-1] = np.average(V[:,tour_i-1])
            V[Z_i][tour_i] += (1 / (np.ceil(tour_i/tour_N)+tour_c))* \
                            (R_k - V_def[tour_i-1]*xi + V[Z_f][tour_i-1] - V[Z_i][tour_i-1])
        elif techq == 3:
            V_def[tour_i-1] = np.min(V[:,tour_i-1])
            V[Z_i][tour_i] += (1 / (np.ceil(tour_i/tour_N)+tour_c))* \
                            (R_k - V_def[tour_i-1]*xi + V[Z_f][tour_i-1] - V[Z_i][tour_i-1])
        else:
            print("Wrong value for RL technique")
            sys.exit(0)
        tour_i += 1
    # return(V_def[1:tour_i-1])
    xi_arrr = np.array(xi_arr)
    ind = np.zeros(len(B_arr),dtype = int)
    for ii,i in enumerate(B_arr):
        ind[ii]=np.argmax(xi_arrr > i)-1
    return (V_def[ind+1])
    # return (V_def[1:tour_i-1], np.array(R)[:tour_i-1], np.array(xi_arr)[:-1])

## RL-RW with Lazy Random Walk

In [45]:
# RL with Lazy Random Walk
def RL_LRW_estimate(G,super_node,max_B,B_arr,G_no_nodes,techq,tour_N,tour_c):
    """
    1. Returns estimate from tours
    2. function  fn: f(D_t) = 1 if D_t > deg_def

    Parameters
    ----------
    G:            - networkx graph
    super_node	  - node set  for "super node"
    no_tours	  - number of tours
    """
    size_super_node = len(super_node)
    V = np.zeros((size_super_node,max_B+1))
    V_def = np.zeros(max_B)
    Z_def = 10
    R=[]
    xi_arr = []
    xi_sum = 0
    tour_i = 1
    
    while xi_sum <= max_B:

        R_k = 0
        Z_i = random.randint(0,size_super_node-1)
        sample = super_node[Z_i]
        
        # sample_t contains the previous sample
        sample_t =  sample
        if not np.random.randint(2,size = 1):
            neighbors = nx.neighbors(G,sample)
            sample = np.random.choice(neighbors)

        xi = 1
        LH = 1
        while sample not in super_node:
            R_k += node_fn(sample)
            
            if sample == sample_t:
                LH_1 = 1-sum([1/max(G.degree(sample_t), G.degree(sample_t_n))\
                              for sample_t_n in neighbors])
                LH_2 = 1/2
            else:
                LH_1 = 1/max(G.degree(sample_t), G.degree(sample))
                LH_2 = 1/(2*G.degree(sample_t))
                
            LH *= LH_1 / LH_2
            
            sample_t = sample
            if not np.random.randint(2,size = 1):
                neighbors = nx.neighbors(G,sample)
                sample = np.random.choice(neighbors)
            xi += 1
            
        R_k += node_fn(sample)
        R.append(R_k)
        xi_sum += xi
        xi_arr.append(xi_sum)
        
        LH_1 = 1/max(G.degree(sample), G.degree(sample_t))
        LH_2 = 1/(2*G.degree(sample_t))
        LH *= LH_1 / LH_2
        
        V[:,tour_i] = V[:,(tour_i-1)]

        Z_f = super_node.index(sample)
        if techq == 1:
            V_def[tour_i-1] = V[Z_def,tour_i-1]
            V[Z_i][tour_i] += LH * (1 / (np.ceil(tour_i/tour_N)+tour_c)) \
                                * (R_k - V_def[tour_i-1]*xi + V[Z_f][tour_i-1] - V[Z_i][tour_i-1])
        elif techq == 2:
            V_def[tour_i-1] = np.average(V[:,tour_i-1])
            V[Z_i][tour_i] +=  LH * (1 / (np.ceil(tour_i/tour_N)+tour_c)) \
                                * (R_k - V_def[tour_i-1]*xi + V[Z_f][tour_i-1] - V[Z_i][tour_i-1])
        elif techq == 3:
            V_def[tour_i-1] = np.min(V[:,tour_i-1])
            V[Z_i][tour_i] += LH * (1 / (np.ceil(tour_i/tour_N)+tour_c)) \
                                * (R_k - V_def[tour_i-1]*xi + V[Z_f][tour_i-1] - V[Z_i][tour_i-1])
        else:
            print("Wrong value for RL technique")
            sys.exit(0)
        tour_i += 1

    xi_arrr = np.array(xi_arr)
    ind = np.zeros(len(B_arr),dtype = int)
    for ii,i in enumerate(B_arr):
        ind[ii]=np.argmax(xi_arrr > i)-1
    return (V_def[ind+1])

## RL-RW with SRW

In [41]:
# RL with Lazy Random Walk
def RL_SRW_estimate(G,super_node,max_B,B_arr,G_no_nodes,techq,tour_N,tour_c):
    """
    1. Returns estimate from tours

    Parameters
    ----------
    G:            - networkx graph
    super_node	  - node set  for "super node"
    no_tours	  - number of tours
    To be completed
    """
    size_super_node = len(super_node)
    V = np.zeros((size_super_node,max_B+1))
    V_def = np.zeros(max_B)
    Z_def = 10
    R=[]
    xi_arr = []
    xi_sum = 0
    tour_i = 1
    while xi_sum <= max_B:

        R_k = 0
        Z_i = random.randint(0,size_super_node-1)
        sample = super_node[Z_i]
        
        # sample_t contains the previous sample
        sample_t =  sample
        neighbors = nx.neighbors(G,sample)
        sample = np.random.choice(neighbors)

        xi = 1
        LH = 1
        while sample not in super_node:
            R_k += node_fn(sample)
            
            LH_1 = 1/max(G.degree(sample_t), G.degree(sample))
            LH_2 = 1/(G.degree(sample_t))
            LH *= LH_1 / LH_2
            
            sample_t = sample
            neighbors = nx.neighbors(G,sample)
            sample = np.random.choice(neighbors)
            xi += 1
            
        R_k += node_fn(sample)
        R.append(R_k)
        xi_sum += xi
        xi_arr.append(xi_sum)
        
        LH_1 = 1/max(G.degree(sample), G.degree(sample_t))
        LH_2 = 1/(G.degree(sample_t))
        LH *= LH_1 / LH_2
              
        V[:,tour_i] = V[:,(tour_i-1)]

        Z_f = super_node.index(sample)
        if techq == 1:
            V_def[tour_i-1] = V[Z_def,tour_i-1]
            V[Z_i][tour_i] += LH * (1 / (np.ceil(tour_i/tour_N)+tour_c)) \
                                * (R_k - V_def[tour_i-1]*xi + V[Z_f][tour_i-1] - V[Z_i][tour_i-1])
        elif techq == 2:
            V_def[tour_i-1] = np.average(V[:,tour_i-1])
            V[Z_i][tour_i] +=  LH * (1 / (np.ceil(tour_i/tour_N)+tour_c)) \
                                * (R_k - V_def[tour_i-1]*xi + V[Z_f][tour_i-1] - V[Z_i][tour_i-1])
        elif techq == 3:
            V_def[tour_i-1] = np.min(V[:,tour_i-1])
            V[Z_i][tour_i] += LH * (1 / (np.ceil(tour_i/tour_N)+tour_c)) \
                                * (R_k - V_def[tour_i-1]*xi + V[Z_f][tour_i-1] - V[Z_i][tour_i-1])
        else:
            print("Wrong value for RL technique")
            sys.exit(0)
        tour_i += 1
        
    xi_arrr = np.array(xi_arr)
    ind = np.zeros(len(B_arr),dtype = int)
    for ii,i in enumerate(B_arr):
        ind[ii]=np.argmax(xi_arrr > i)-1
    return V_def[ind+1]
    # return (V_def[1:tour_i-1], np.array(R)[:tour_i-1], np.array(xi_arr)[:-1])

## Tour estimator

In [42]:
def random_walk_tour_estimate(G,super_node,nbr_out_sup,F_org_sup_1,vol_sup_node,max_B,B_arr):
    """
    1. Returns estimate from tours
    2. function  fn: f(D_t) = 1 if D_t > deg_def

    Parameters
    ----------
    G:            - networkx graph
    super_node	  - node set  for "super node"
    no_tours	  - number of tours
    """

    R = []
    R_1 =[]
    R_2 = []
    summ = 0
    summ1 =0
    summ2 =0
    
    xi_arr = []
    xi_sum = 0

    size_super_node = len(super_node)
    tour_i = 1
    while xi_sum <= max_B:

        R_k1 = 0
        R_k2 = 0
        
        sample = random.choice(nbr_out_sup)
        xi = 1
        while sample not in super_node:
            deg_sample = G.degree(sample)
            R_k1 += node_fn(sample)/deg_sample
            R_k2 += 1/deg_sample
            xi += 1
            neighbors = nx.neighbors(G,sample)
            sample = random.choice(neighbors)

        R_k1 += F_org_sup_1/vol_sup_node
        R_k2 += size_super_node/vol_sup_node
        
        R_1.append(R_k1+summ1)
        R_2.append(R_k2+summ2)
        
        summ1 += R_k1
        summ2 += R_k2
        
        xi_sum += xi
        xi_arr.append(xi_sum)
        tour_i += 1
#     R_1 = np.array(R_1)+F_org_sup_1
#     R_2 = np.array(R_2)+len(super_node)

    R_1 = np.array(R_1)
    R_2 = np.array(R_2)
    R = R_1/R_2
    
    # xi_arrr = np.array(xi_arr)
    # ind = np.zeros(len(B_arr),dtype = int)
    # for ii,i in enumerate(B_arr):
    #     ind[ii]=np.argmax(xi_arrr > i)-1
    # return (R[ind])
    
    xi_arrr = np.array(xi_arr)
    ind = np.zeros(len(B_arr),dtype = int)
    for ii,it in enumerate(B_arr):
        ind[ii]=np.argmax(xi_arrr > it)-1 
        # Without -1, argmax chooses the xi_arrth element which crosses "it"
        # With -1, we go one step backwards
    return (R[ind])

# Main program

In [43]:
if G_i == 1:
    G = nx.read_edgelist("friendster_community1_trimmed.edgelist",nodetype = int)
elif G_i == 2:
    G = nx.read_edgelist("lesmis.edgelist", nodetype = int)
G_no_edges=G.number_of_edges()
G_no_nodes=G.number_of_nodes()

In [44]:
if G_i == 1:
    if flag_fn == 1:
        F_org = 0.265712074303 #sum([1 for i in G.nodes() if G.degree(i) ])/G_no_nodes
    elif flag_fn == 4:
        F_org = 0.4491010966748313
    else:
        sys.exit("Invalid function")
        
elif G_i == 2:
    # F_org = sum([1 for i in G.nodes() if G.degree(i) ])/G_no_nodes #0.220779220779 #
    if (flag_fn == 1) or (flag_fn == 2):
        F_org = sum([node_fn(i) for i in G.nodes()])/G_no_nodes
    elif flag_fn == 3:
        F_org = 2*G_no_edges/G_no_nodes
    elif flag_fn == 4:
        F_org = nx.average_clustering(G)

In [16]:
if MH_do:
    MSE_MH_t = 0
    for ii in range(1,no_runs+1):
        print("MHruns: ",ii)
        MSE_MH_t += (MH_sampling(G,max_B)-F_org)**2
    MSE_MH = MSE_MH_t/(no_runs)

    if SHOW_VAR_CONVG == 1:
        MSE_MH = MSE_MH*np.arange(1,max_B+1)
    else:
        MSE_MH = np.sqrt(MSE_MH)/F_org

In [17]:
if RDS_do:
    MSE_rds_t = 0
    for ii in range(1,no_runs+1):
        print("RDSruns: ",ii)
        MSE_rds_t += (RDS_sampling(G,max_B)-F_org)**2
    MSE_rds = MSE_rds_t/(no_runs)

    if SHOW_VAR_CONVG == 1:
        MSE_rds = MSE_rds*np.arange(1,max_B+1)
    else:
        MSE_rds = np.sqrt(MSE_rds)/F_org

RDSruns:  1
RDSruns:  2
RDSruns:  3
RDSruns:  4
RDSruns:  5
RDSruns:  6
RDSruns:  7
RDSruns:  8
RDSruns:  9
RDSruns:  10
RDSruns:  11
RDSruns:  12
RDSruns:  13
RDSruns:  14
RDSruns:  15
RDSruns:  16
RDSruns:  17
RDSruns:  18
RDSruns:  19
RDSruns:  20
RDSruns:  21
RDSruns:  22
RDSruns:  23
RDSruns:  24
RDSruns:  25
RDSruns:  26
RDSruns:  27
RDSruns:  28
RDSruns:  29
RDSruns:  30
RDSruns:  31
RDSruns:  32
RDSruns:  33
RDSruns:  34
RDSruns:  35
RDSruns:  36
RDSruns:  37
RDSruns:  38
RDSruns:  39
RDSruns:  40
RDSruns:  41
RDSruns:  42
RDSruns:  43
RDSruns:  44
RDSruns:  45
RDSruns:  46
RDSruns:  47
RDSruns:  48
RDSruns:  49
RDSruns:  50
RDSruns:  51
RDSruns:  52
RDSruns:  53
RDSruns:  54
RDSruns:  55
RDSruns:  56
RDSruns:  57
RDSruns:  58
RDSruns:  59
RDSruns:  60
RDSruns:  61
RDSruns:  62
RDSruns:  63
RDSruns:  64
RDSruns:  65
RDSruns:  66
RDSruns:  67
RDSruns:  68
RDSruns:  69
RDSruns:  70
RDSruns:  71
RDSruns:  72
RDSruns:  73
RDSruns:  74
RDSruns:  75
RDSruns:  76
RDSruns:  77
RDSruns:

In [18]:
if RL_MH_do:
    ## Super-node calculations =================================================
    size_super_node = int(input("Size of super node (def = 1000): ") or "1000")
    if size_super_node == 1:
        sup_node_1_sel = input("Select highest degree node (y/n), def = y): ") or "y"
        if sup_node_1_sel is 'y':
            super_node = [max(G.degree(), key=G.degree().get)]
            print("Selected the node with highest degree !!")
        else:
            super_node = random.sample(G.nodes(), size_super_node) #super-node formation from uniform samples
    if size_super_node is not 1:
        super_node = random.sample(G.nodes(), size_super_node) #super-node formation from uniform samples
        print("Size of super node is not 1")
    ## =========================================================================
    B_arr = np.arange(100,max_B+1,100)

    for ii in range(1,no_runs+1):
        print("RLruns: ",ii)
        temp = RL_estimate_MH(G,super_node,max_B,B_arr,G_no_nodes, 2,tour_N,tour_c)
        MSE_RL_t = (temp-F_org)**2
        if ii == 1:
            MSE_RL = MSE_RL_t
        else:
            MSE_RL = ((ii-1)/ii)*MSE_RL + MSE_RL_t/ii

    if SHOW_VAR_CONVG == 1:
        MSE_RL = MSE_RL*B_arr
    else:
        MSE_RL = np.sqrt(MSE_RL)/F_org

Size of super node (def = 1000): 25
Size of super node is not 1
RLruns:  1
RLruns:  2
RLruns:  3
RLruns:  4
RLruns:  5
RLruns:  6


KeyboardInterrupt: 

In [ ]:
MSE_RL

## RL-RW

In [46]:
if RL_RW_do:
    ## Super-node calculations =================================================
    size_super_node = int(input("Size of super node (def = 1000): ") or "1000")
    if size_super_node == 1:
        sup_node_1_sel = input("Select highest degree node (y/n), def = y): ") or "y"
        if sup_node_1_sel is 'y':
            super_node = [max(G.degree(), key=G.degree().get)]
            print("Selected the node with highest degree !!")
        else:
            super_node = random.sample(G.nodes(), size_super_node) #super-node formation from uniform samples
    if size_super_node is not 1:
        super_node = random.sample(G.nodes(), size_super_node) #super-node formation from uniform samples
        print("Size of super node is not 1")
    ## =========================================================================
    B_arr = np.arange(100,max_B+1,100)

    for ii in range(1,no_runs+1):
        print("RLruns: ",ii)
#         temp = RL_estimate_RW(G,super_node,max_B,B_arr,G_no_nodes, 2,tour_N,tour_c)
        temp = RL_LRW_estimate(G = G,super_node = super_node, max_B = max_B ,B_arr = B_arr ,
                                  G_no_nodes = G_no_nodes ,techq = 2,tour_N = tour_N ,tour_c = tour_c)
        MSE_RL_RW_t = (temp-F_org)**2
        if ii == 1:
            MSE_RL_RW = MSE_RL_RW_t
        else:
            MSE_RL_RW = ((ii-1)/ii)*MSE_RL_RW + MSE_RL_RW_t/ii

    if SHOW_VAR_CONVG == 1:
        MSE_RL_RW = MSE_RL_RW*B_arr
    else:
        MSE_RL_RW = np.sqrt(MSE_RL_RW)/F_org 

Size of super node (def = 1000): 1000
Size of super node is not 1
RLruns:  1
RLruns:  2
RLruns:  3
RLruns:  4
RLruns:  5
RLruns:  6
RLruns:  7
RLruns:  8
RLruns:  9
RLruns:  10
RLruns:  11
RLruns:  12
RLruns:  13
RLruns:  14
RLruns:  15
RLruns:  16
RLruns:  17
RLruns:  18
RLruns:  19
RLruns:  20
RLruns:  21
RLruns:  22
RLruns:  23
RLruns:  24
RLruns:  25
RLruns:  26
RLruns:  27
RLruns:  28
RLruns:  29
RLruns:  30
RLruns:  31
RLruns:  32
RLruns:  33
RLruns:  34
RLruns:  35
RLruns:  36
RLruns:  37
RLruns:  38
RLruns:  39
RLruns:  40
RLruns:  41
RLruns:  42
RLruns:  43
RLruns:  44
RLruns:  45
RLruns:  46
RLruns:  47
RLruns:  48
RLruns:  49
RLruns:  50
RLruns:  51
RLruns:  52
RLruns:  53
RLruns:  54
RLruns:  55
RLruns:  56
RLruns:  57
RLruns:  58
RLruns:  59
RLruns:  60
RLruns:  61
RLruns:  62
RLruns:  63
RLruns:  64
RLruns:  65
RLruns:  66
RLruns:  67
RLruns:  68
RLruns:  69
RLruns:  70
RLruns:  71
RLruns:  72
RLruns:  73
RLruns:  74
RLruns:  75
RLruns:  76
RLruns:  77
RLruns:  78
RLruns:

In [33]:
MSE_RL_RW

array([  1.30797680e+07,   1.30798959e+07,   1.30798953e+07,
         2.19500233e+13,   7.34023345e+13,   7.34023352e+13,
         2.03757385e+14,   2.03757353e+14,   2.03757231e+14,
         2.03757315e+14,   2.03757319e+14,   2.03762168e+14,
         2.03749486e+14,   2.03749602e+14,   2.03749601e+14,
         2.03749603e+14,   2.03749453e+14,   2.03758736e+14,
         6.84720954e+17,   6.84261231e+17,   6.84261065e+17,
         6.84262691e+17,   6.84262668e+17,   6.84262745e+17,
         6.84262662e+17,   6.84286346e+17,   6.84286323e+17,
         6.84286408e+17,   6.84289665e+17,   6.84985710e+17,
         6.84985750e+17,   6.84985702e+17,   6.84985954e+17,
         6.84914584e+17,   6.84928877e+17,   8.04218306e+17,
         3.46497634e+18,   5.00469289e+18,   1.70365160e+19,
         1.70365253e+19,   1.71532040e+19,   1.71532060e+19,
         1.72004438e+19,   1.72011953e+19,   1.72012016e+19,
         1.72011970e+19,   1.72007944e+19,   4.51124141e+23,
         4.68570321e+23,

## Ratio estimator comparison

In [ ]:
## Super-node calculations =================================================
size_super_node = int(input("Size of super node (def = 1000): ") or "1000")
if size_super_node == 1:
    sup_node_1_sel = input("Select highest degree node (y/n), def = y): ") or "y"
    if sup_node_1_sel is 'y':
        super_node = [max(G.degree(), key=G.degree().get)]
        print("Selected the node with highest degree !!")
    else:
        super_node = random.sample(G.nodes(), size_super_node) #super-node formation from uniform samples
if size_super_node is not 1:
    super_node = random.sample(G.nodes(), size_super_node) #super-node formation from uniform samples
    print ("Size of super node is not 1")
## =========================================================================
#Finding super_node's neighbours outside super node
nbr_out_sup=[]
sum_temp=0
F_org_sup_1=0
i=0
for node_s in super_node:
    print("super node ", i)
    i+=1
    list_s_n=[node_s_n for node_s_n in nx.neighbors(G,node_s) if node_s_n not in super_node]
    nbr_out_sup=nbr_out_sup+list_s_n
    F_org_sup_1 += node_fn(node_s)

vol_sup_node = len(nbr_out_sup)
## ==========================================================================

In [ ]:
B_arr = np.arange(100,max_B+1,100)
MSE_RL_MH_t = 0
MSE_RL_RW_t = 0
MSE_tour_ratio_t = 0
MSE_rds_t = 0

for ii in range(1,no_runs+1):
    print("runs: ",ii)
#     temp_RL_MH = RL_estimate_MH(G,super_node,max_B,B_arr,G_no_nodes, 1,tour_N,tour_c)
#     temp_RL_RW = RL_estimate_RW(G,super_node,max_B,B_arr,G_no_nodes, 1,tour_N,tour_c)
    temp_tour_ratio = random_walk_tour_estimate(G,super_node,nbr_out_sup,F_org_sup_1,vol_sup_node,max_B,B_arr)
    
#     MSE_RL_MH_t = (temp_RL_MH-F_org)**2
#     MSE_RL_RW_t = (temp_RL_RW-F_org)**2
    MSE_tour_ratio_t = (temp_tour_ratio-F_org)**2
    MSE_rds_t += (RDS_sampling(G,max_B)-F_org)**2

    if ii == 1:
#         MSE_RL_MH = MSE_RL_MH_t
#         MSE_RL_RW = MSE_RL_RW_t
        MSE_tour_ratio = MSE_tour_ratio_t
    else:
#         MSE_RL_MH = ((ii-1)/ii)*MSE_RL_MH + MSE_RL_MH_t/ii
#         MSE_RL_RW = ((ii-1)/ii)*MSE_RL_RW + MSE_RL_RW_t/ii
        MSE_tour_ratio = ((ii-1)/ii)*MSE_tour_ratio + MSE_tour_ratio_t/ii
MSE_rds = MSE_rds_t/(no_runs)
        
# if SHOW_VAR_CONVG == 1:
#     MSE_RL_RW = MSE_RL_RW*B_arr
# else:
#     MSE_RL_RW = np.sqrt(MSE_RL_RW)/F_org 

# Plotting and Saving

## Temporary experiments with tour based ratio estimator

In [ ]:
import brewer2mpl

# Get "Set2" colors from ColorBrewer (all colorbrewer scales: http://bl.ocks.org/mbostock/5577023)
set2 = brewer2mpl.get_map('Set2', 'qualitative', 8).mpl_colors
# plt.style.use('default')


In [ ]:
FIGSIZE_REQD = 1
# fig = plt.figure(figsize=figsize(FIGSIZE_REQD))
fig, ax = plt.subplots(FIGSIZE_REQD)

ax=plt.axes(frameon=1)
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)

# Real plotting goes here:

if SHOW_VAR_CONVG == 1:
    plt.plot(np.arange(11,max_B+1),MSE_MH[10:], color=cust_color[1], \
     linewidth=1,linestyle='-',label='MH-MCMC')
    if (G_i == 2):
        plt.axhline(asy_var_MH_lesmis[flag_fn-1], xmin=0, color=cust_color[1],alpha=0.5, \
        linewidth=1.35, linestyle="--", label = 'Asymp. variance of MH-MCMC') #Plot asymptotic variance
    
    plt.plot(range(11,max_B+1),MSE_rds[10:], color=cust_color[6], \
     linewidth=1,linestyle='-',label='RDS')
    if (G_i == 2):
        plt.axhline(asy_var_RDS_lesmis[flag_fn-1], xmin=0, color=cust_color[6],alpha=0.5, \
        linewidth=1.35, linestyle=":", label = 'Asymp. variance of RDS') #Plot asymptotic variance

    plt.plot(B_arr[1:],MSE_RL[1:], color=cust_color[3], \
     linewidth=1,linestyle='-',label='RL technique')

else:
#     plt.plot(np.arange(11,max_B+1),MSE_MH[10:], color=cust_color[1], \
#      linewidth=1,linestyle='-',label='MH-MCMC')      

#     plt.plot(range(11,max_B+1),MSE_rds[10:], color=cust_color[6], \
#      linewidth=1,linestyle='-',label='RDS')

#     plt.plot(B_arr,MSE_RL, color=cust_color[3], \
#      linewidth=1,linestyle='-',label='RL technique MH')
        plt.plot(B_arr,MSE_tour_ratio_fr_average_clust_coeff_sup_1000_save, color= cust_color[1], \
        linewidth=1,linestyle='-',label='Tour technique')
#         plt.plot(B_arr,MSE_RL_MH, color=cust_color[4], \
#         linewidth=1,linestyle='-',label='RL MH')
#         plt.plot(B_arr,MSE_RL_RW, color=cust_color[5], \
#         linewidth=1,linestyle='-',label='RL RW')
        plt.plot(range(11,max_B+1),MSE_rds_fr_avg_clust_coeff_save[10:], color= cust_color[6], \
         linewidth=1,linestyle='-',label='RDS')

    
#     plt.plot(B_arr,MSE_RL_RW, color=cust_color[4], \
#      linewidth=1,linestyle='-',label='RL technique RW')


# =======================

ax.set_xlabel('Budget $B$')
if SHOW_VAR_CONVG == 1:
    ylabel_txt = r'$\mbox{MSE}\times B$'
else:
    ylabel_txt = 'NRMSE'

ax.set_ylabel(ylabel_txt) #USE labelpad = -1 to move label to axis

legend=ax.legend(fancybox = True, framealpha=0.5,loc='best')

frame = legend.get_frame()
frame.set_facecolor('0.9')
frame.set_edgecolor('0.75')
plt.grid(alpha = 0.7)

# props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
# textstr = "Super node size =%d"%(size_super_node)
# ax.text(0.02, 0.95,textstr, transform=ax.transAxes, fontsize= 8,\
#         verticalalignment='top', bbox=props)

### QUICK INSET SETTINGS ####
ax2 = plt.axes([.6, .3, .3, .3]) #[left bottom width height]
plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
plt.grid(alpha = 0.7)

plt.plot(B_arr,MSE_tour_ratio_fr_average_clust_coeff_sup_1000_save, color=cust_color[1], \
linewidth=1,linestyle='-')
plt.plot(range(11,max_B+1),MSE_rds_fr_avg_clust_coeff_save[10:], color=cust_color[6], \
 linewidth=1,linestyle='-',label='RDS')
ax2.set_xlim(left = 2000)
ax2.set_ylim(top = 0.001)

plt.locator_params(axis='y',nbins=4)#to specify number of ticks on both or any single axes
plt.locator_params(axis='x',nbins=8)
try:
    legend=ax2.legend(fancybox = True, framealpha=0.5,loc='best')
    frame = legend.get_frame()
    frame.set_facecolor('0.9')
    frame.set_edgecolor('0.7')
except:
    pass
### -------------------- ####

props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
textstr = r'Friendster graph: \\Estimate avg.\ clustering coefficient'
# textstr = r'Friendster graph:\\ Estimate $\displaystyle \frac{1}{\lvert V\rvert} \sum_{u \in V}\mathbb{I}\{d(u) > 50\}$'
# textstr = r'Les Miserables graph: Estimate $\displaystyle \frac{1}{\lvert V\rvert} \sum_{u \in V} d(u)$'
# textstr = r'Les Miserables graph: Estimate avg.\ clustering coefficient'


plt.text(0.05, 0.7,textstr,transform=ax.transAxes, bbox=props)

if SHOW_VAR_CONVG == 1:
    plt.savefig('plot_estn_comp_csonet_var.pdf',bbox_inches='tight',pad_inches = 0.05)
    pl.dump(ax,file('plot_estn_comp_csonet_var.pickle','w'))
    print("Saved the file plot_estn_comp_csonet_var")
else:
    plt.savefig('plot_estn_comp_csonet_mse.pdf',bbox_inches='tight',pad_inches = 0.05)
    pl.dump(ax2,open('plot_estn_comp_csonet_mse.pickle','wb'))
    pickle.dump(fig, open('myplot.pickle', 'wb')) #ax corresponds to fig, ax = plt.subplots()

    print("Saved the file plot_estn_comp_csonet_mse")

In [ ]:
data_friendster_rds; data_friendster_rt

In [ ]:
store_friendster = pd.HDFStore('data_friendster.h5')
store_friendster['data_friendster_rds'] = data_friendster_rds
store_friendster['data_friendster_rt'] = data_friendster_rt
store_friendster.close()

In [ ]:
## Plotting without INSET
FIGSIZE_REQD = 0.5
# fig = plt.figure(figsize=figsize(FIGSIZE_REQD))
fig, ax = plt.subplots(figsize=figsize(FIGSIZE_REQD))

ax=plt.axes(frameon=1)
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)

# Real plotting goes here:
ax.plot(B_arr,MSE_tour_ratio, color=cust_color[1], \
linewidth=1.5,linestyle='-',label='R-T')
# ax.plot(B_arr,MSE_tour_ratio_1000, color=cust_color[2], \
# linewidth=1.5,linestyle='-',label='R-T: super-node size = 1000')
ax.plot(range(11,max_B+1),MSE_rds[10:], color=cust_color[6], \
 linewidth=1.5,linestyle='-', label='RDS')
# =======================

ax.set_xlabel('Budget $B$')
ylabel_txt = 'NRMSE'
ax.set_ylabel(ylabel_txt) #USE labelpad = -1 to move label to axis
legend=ax.legend(fancybox = True, framealpha=0.5,loc='best')
# legend=ax.legend(fancybox = True, framealpha=0.5,loc='best')

frame = legend.get_frame()
frame.set_facecolor('0.9')
frame.set_edgecolor('0.75')
plt.grid(alpha = 0.7)


# ax.set_xlim(left = 2000)
# ax.set_ylim(top = 0.0011)

plt.locator_params(axis='y',nbins=6)#to specify number of ticks on both or any single axes
plt.locator_params(axis='x',nbins=4)

# props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
# textstr = "Super node size =%d"%(size_super_node)
# ax.text(0.02, 0.95,textstr, transform=ax.transAxes, fontsize= 8,\
#         verticalalignment='top', bbox=props)

try:
    legend=ax.legend(fancybox = True, framealpha=0.5,loc='best')
    frame = legend.get_frame()
    frame.set_facecolor('0.9')
    frame.set_edgecolor('0.7')
except:
    pass
### -------------------- ####


plt.savefig('plot_estn_comp_csonet_mse.pdf',bbox_inches='tight',pad_inches = 0.05)
# pl.dump(ax2,open('plot_estn_comp_csonet_mse.pickle','wb'))
#     pickle.dump(ax, open('myplot.pickle', 'wb')) #ax corresponds to fig, ax = plt.subplots()

print("Saved the file plot_estn_comp_csonet_mse")

In [ ]:
ax11 = pl.load(open('plot_estn_comp_csonet_mse.pickle', 'rb'))
# ax1 = pickle.load(open('myplot.pickle', 'rb'))

## Temporary experiments with RL-RW

In [ ]:
MSE_RL_RW

In [ ]:
FIGSIZE_REQD = 1
fig = plt.figure(figsize=figsize(FIGSIZE_REQD))
ax=plt.axes(frameon=1)
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)

# Real plotting goes here:

#     plt.plot(np.arange(11,max_B+1),MSE_MH[10:], color=cust_color[1], \
#      linewidth=1,linestyle='-',label='MH-MCMC')      

# plt.plot(range(11,max_B+1),MSE_rds[10:], color=cust_color[6], \
#  linewidth=1,linestyle='-',label='RDS')

#     plt.plot(B_arr,MSE_RL, color=cust_color[3], \
#      linewidth=1,linestyle='-',label='RL technique MH')

plt.plot(B_arr[:-50],MSE_RL_RW[:-50], color=cust_color[4], \
 linewidth=1,linestyle='-',label='RL technique RW')


# =======================

ax.set_xlabel('Budget $B$')
if SHOW_VAR_CONVG == 1:
    ylabel_txt = r'$\mbox{MSE}\times B$'
else:
    ylabel_txt = 'NRMSE'

ax.set_ylabel(ylabel_txt) #USE labelpad = -1 to move label to axis

legend=ax.legend(fancybox = True, framealpha=0.5,loc='best')

frame = legend.get_frame()
frame.set_facecolor('0.9')
frame.set_edgecolor('0.75')
plt.grid(alpha = 0.7)

# props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
# textstr = "Super node size =%d"%(size_super_node)
# ax.text(0.02, 0.95,textstr, transform=ax.transAxes, fontsize= 8,\
#         verticalalignment='top', bbox=props)

# ### QUICK INSET SETTINGS ####
# ax2 = plt.axes([.6, .3, .3, .3]) #[left bottom width height]
# plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
# plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
# plt.grid(alpha = 0.7)
# plt.plot(B_arr[:-100],MSE_RL_RW[:-100], color=cust_color[4], \
#  linewidth=1,linestyle='-',label='RL technique RW')
# plt.locator_params(axis='y',nbins=4)#to specify number of ticks on both or any single axes
# plt.locator_params(axis='x',nbins=8)
# legend=ax2.legend(fancybox = True, framealpha=0.5,loc='best')
# frame = legend.get_frame()
# frame.set_facecolor('0.9')
# frame.set_edgecolor('0.75')
# ### -------------------- ####


plt.savefig('plot_estn_comp_csonet_mse.png',bbox_inches='tight',pad_inches = 0.05)
#     pl.dump(fig,file('plot_estn_comp_csonet_mse.pickle','w'))
print("Saved the file plot_estn_comp_csonet_mse")

In [ ]:
MSE_RL_RW

In [ ]:
FIGSIZE_REQD = 1
fig = plt.figure(figsize=figsize(FIGSIZE_REQD))
ax=plt.axes(frameon=1)
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)

# Real plotting goes here:

if SHOW_VAR_CONVG == 1:
    plt.plot(np.arange(11,max_B+1),MSE_MH[10:], color=cust_color[1], \
     linewidth=1,linestyle='-',label='MH-MCMC')
    if (G_i == 2):
        plt.axhline(asy_var_MH_lesmis[flag_fn-1], xmin=0, color=cust_color[1],alpha=0.5, \
        linewidth=1.35, linestyle="--", label = 'Asymp. variance of MH-MCMC') #Plot asymptotic variance
    
    plt.plot(range(11,max_B+1),MSE_rds[10:], color=cust_color[6], \
     linewidth=1,linestyle='-',label='RDS')
    if (G_i == 2):
        plt.axhline(asy_var_RDS_lesmis[flag_fn-1], xmin=0, color=cust_color[6],alpha=0.5, \
        linewidth=1.35, linestyle="--", label = 'Asymp. variance of RDS') #Plot asymptotic variance

    plt.plot(B_arr[1:],MSE_RL[1:], color=cust_color[3], \
     linewidth=1,linestyle='-',label='RL technique')

else:
    plt.plot(np.arange(11,max_B+1),MSE_MH[10:], color=cust_color[1], \
     linewidth=1,linestyle='-',label='MH-MCMC')      

    plt.plot(range(11,max_B+1),MSE_rds[10:], color=cust_color[6], \
     linewidth=1,linestyle='-',label='RDS')

    plt.plot(B_arr,MSE_RL, color=cust_color[3], \
     linewidth=1,linestyle='-',label='RL technique')
    
  
# =======================

ax.set_xlabel('Budget $B$')
if SHOW_VAR_CONVG == 1:
    ylabel_txt = r'$\mbox{MSE}\times B$'
else:
    ylabel_txt = 'NRMSE'

ax.set_ylabel(ylabel_txt) #USE labelpad = -1 to move label to axis

legend=ax.legend(fancybox = True, framealpha=0.5,loc='best')

frame = legend.get_frame()
frame.set_facecolor('0.9')
frame.set_edgecolor('0.75')

# props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
# textstr = "Super node size =%d"%(size_super_node)
# ax.text(0.02, 0.95,textstr, transform=ax.transAxes, fontsize= 8,\
#         verticalalignment='top', bbox=props)

plt.grid(alpha = 0.7)
if SHOW_VAR_CONVG == 1:
    plt.savefig('plot_estn_comp_csonet_var.pdf',bbox_inches='tight',pad_inches = 0.05)
    pl.dump(fig,file('plot_estn_comp_csonet_var.pickle','w'))
    print "Saved the file plot_estn_comp_csonet_var"
else:
    plt.savefig('plot_estn_comp_csonet_mse.png',bbox_inches='tight',pad_inches = 0.05)
    pl.dump(fig,file('plot_estn_comp_csonet_mse.pickle','w'))
    print "Saved the file plot_estn_comp_csonet_mse"

#tour_N = 10, tour_c = 0
MSE_RL_Le_3_var = np.copy(MSE_RL)
MSE_rds_Le_3_var = np.copy(MSE_rds)
MSE_MH_Le_3_var = np.copy(MSE_MH)

#tour_N = 10, tour_c = 0
MSE_RL_Le_2_var = np.copy(MSE_RL)
MSE_rds_Le_2_var = np.copy(MSE_rds)
MSE_MH_Le_2_var = np.copy(MSE_MH)

#tour_N = 10, tour_c = 0
MSE_RL_Le_1_var = np.copy(MSE_RL)
MSE_rds_Le_1_var = np.copy(MSE_rds)
MSE_MH_Le_1_var = np.copy(MSE_MH)

#tour_N = 10, tour_c = 0
MSE_RL_Le_4 = np.copy(MSE_RL)
MSE_rds_Le_4 = np.copy(MSE_rds)
MSE_MH_Le_4 = np.copy(MSE_MH)

#tour_N = 10, tour_c = 0
MSE_RL_Le_3 = np.copy(MSE_RL)
MSE_rds_Le_3 = np.copy(MSE_rds)
MSE_MH_Le_3 = np.copy(MSE_MH)

#tour_N = 10, tour_c = 0
MSE_RL_Le_2 = np.copy(MSE_RL)
MSE_rds_Le_2 = np.copy(MSE_rds)
MSE_MH_Le_2 = np.copy(MSE_MH)

#tour_N = 10, tour_c = 0
MSE_RL_Le_1 = np.copy(MSE_RL)
MSE_rds_Le_1 = np.copy(MSE_rds)
MSE_MH_Le_1 = np.copy(MSE_MH)

#tour_N = 25, tour_c = 0
MSE_RL_Fr_4 = np.copy(MSE_RL)
MSE_rds_Fr_4 = np.copy(MSE_rds)
MSE_MH_Fr_4 = np.copy(MSE_MH)

#tour_N = 25, tour_c = 0
MSE_RL_Fr_1 = np.copy(MSE_RL)
MSE_rds_Fr_1 = np.copy(MSE_rds)
MSE_MH_Fr_1 = np.copy(MSE_MH)

In [ ]:
#tour_N = 10, tour_c = 0
MSE_RL = np.copy(MSE_RL_Le_1)
MSE_rds = np.copy(MSE_rds_Le_1)
MSE_MH = np.copy(MSE_MH_Le_1)

In [ ]:
if SHOW_VAR_CONVG == 1:
    MSE_MH = (MSE_MH*F_org)**2
    MSE_rds = (MSE_rds*F_org)**2
    MSE_RL = (MSE_RL*F_org)**2
    
    MSE_MH = MSE_MH*np.arange(1,max_B+1)
    MSE_rds = MSE_rds*np.arange(1,max_B+1)    
    MSE_RL = MSE_RL*B_arr

In [ ]:
import pickle

In [ ]:
## To write:
# with open('csonet_submission_final.pickle','wb') as handle:
#     pickle.dump((MSE_RL_Le_3_var,MSE_rds_Le_3_var,MSE_MH_Le_3_var,MSE_RL_Le_2_var,MSE_rds_Le_2_var,MSE_MH_Le_2_var,\
#                 MSE_RL_Le_1_var,MSE_rds_Le_1_var,MSE_MH_Le_1_var,MSE_RL_Le_4,MSE_rds_Le_4,MSE_MH_Le_4,MSE_RL_Le_3,\
#                 MSE_rds_Le_3,MSE_MH_Le_3, MSE_RL_Le_2,MSE_rds_Le_2,MSE_MH_Le_2,MSE_RL_Le_1,MSE_rds_Le_1,MSE_MH_Le_1,\
#                 MSE_RL_Fr_4,MSE_rds_Fr_4,MSE_MH_Fr_4,MSE_RL_Fr_1,MSE_rds_Fr_1,MSE_MH_Fr_1),handle)
    
# To restore:    
with open('csonet_submission_final.pickle','rb') as handle:
    (MSE_RL_Le_3_var,MSE_rds_Le_3_var,MSE_MH_Le_3_var,MSE_RL_Le_2_var,MSE_rds_Le_2_var,MSE_MH_Le_2_var,\
    MSE_RL_Le_1_var,MSE_rds_Le_1_var,MSE_MH_Le_1_var,MSE_RL_Le_4,MSE_rds_Le_4,MSE_MH_Le_4,MSE_RL_Le_3,\
    MSE_rds_Le_3,MSE_MH_Le_3, MSE_RL_Le_2,MSE_rds_Le_2,MSE_MH_Le_2,MSE_RL_Le_1,MSE_rds_Le_1,MSE_MH_Le_1,\
    MSE_RL_Fr_4,MSE_rds_Fr_4,MSE_MH_Fr_4,MSE_RL_Fr_1,MSE_rds_Fr_1,MSE_MH_Fr_1) = pickle.load(handle)        


# Miscellaneous